<a href="https://colab.research.google.com/github/custom-hyper/DeFi_Lama_API/blob/master/DeFi_Lama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Setup

In [9]:
#@title  Install  Packages { display-mode: "code" }
!pip install requests
!pip install json
!pip install psycopg2
!pip install pandas


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [2]:
#@title Connect to Drive { display-mode: "code" }
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#@title  Import Libraries { display-mode: "code" }
import requests
import json
from google.colab import data_table
import sqlite3 as sq
import pandas as pd
import logging
import pandas as pd
import logging
import time
from tqdm.notebook import tqdm
from datetime import datetime
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [4]:
#@title  Connect { display-mode: "code" }
path = '/content/drive/MyDrive/workspace/pipelines/db/' #@param {type: 'string'
database = "db.sqlite" #@param {type: 'string'
conn_SQLite = sq.connect(path+database)

#@title   Show Available Tables { display-mode: "code" }
cursor = conn_SQLite.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.execute("DROP TABLE deFi_lama_general;")
cursor.fetchall()

[]

# Data Extraction

Please use  df['slug'] as primary key if loaded into SQL light

## Protocols ID List

In [5]:
#@title Call API { display-mode: "code" }



#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
DeFi = json.loads(output)
full_talbe = pd.json_normalize(DeFi)
full_talbe['id'] = full_talbe['symbol']  + '.' + full_talbe['name']
full_talbe['id'].replace(' ','')
full_talbe['Notes'] = ''

#UNIX timestamp: created
full_talbe['UNIX'] = int( time.time())



full_talbe.head(1)

,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,chainTvls.zkSync,chainTvls.zkSync-offers,chainTvls.Metis-pool2,chainTvls.Harmony-Treasury,chainTvls.Metis-Treasury,chainTvls.Velas-staking,chainTvls.Celo-Treasury,chainTvls.Ergo-staking,Notes,UNIX
0,CRV.Curve,Curve,0xD533a949740bb3306d119CC777fa900bA034cd52,CRV,https://curve.fi,Curve is a decentralized exchange liquidity po...,Multi-Chain,https://icons.llama.fi/curve.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650753273


In [14]:
#@title Save To Directory{ display-mode: "code" }
table = "deFi_lama_general"
sfile = "deFi_lama_general.csv"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
full_talbe.to_csv(path + sfile)

In [15]:
#@title Load to SQLite { display-mode: "code" }

full_talbe.to_sql(table, conn_SQLite , if_exists='append', index=False) 

df = pd.read_sql('select * from {}'.format(sfile), conn_SQLite)


InterfaceError: ignored

In [ ]:
#@title Show Stats { display-mode: "code" }
full_talbe.info()

 ## TVL Timeseries

In [ ]:
#@title Call API  { display-mode: "code" }

#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
table = json.loads(output)
df = pd.json_normalize(table)
slug = df['slug']

path = "https://api.llama.fi/protocol/"
frames = []


try:
  for i in tqdm(slug):
    response = requests.get(path + i)
    output = response.text
    DeFi = json.loads(output)

    full_table = pd.json_normalize(DeFi['tvl']) 
    full_table['slug'] = i
    

    #results = [full_table]
    frames.append(full_table)

    #Slow down requests
    
    time.sleep(60/50)
    #Status report
    
    
except ConnectionError:
    time.sleep(60)


single = pd.concat(frames) #Careful, this line must be outside of the for loop
#single['date_Standard']  = datetime.fromtimestamp(time).strftime('%Y-%m-%d')


single.head()

  


In [ ]:
#@title Save To Directory{ display-mode: "code" }
sfile = "deFi_lama_tvl"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
single.to_csv(path + sfile)

In [ ]:
#@title Load to SQLite { display-mode: "code" }

table = "deFi_lama_tvl"

single.to_sql(table, conn_SQLite , if_exists='replace', index=False) # writes to file

 # Test Connection
df = pd.read_sql('select * from {}'.format(table), conn_SQLite)

df.head()


In [ ]:
#@title Show Stats { display-mode: "code" }
single.info()

## Protocol General Information Data

In [ ]:
#@title Call API  { display-mode: "code" }


#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
DeFi = json.loads(output)
full_talbe = pd.json_normalize(DeFi)
full_talbe['id'] = full_talbe['symbol']  + '.' + full_talbe['name']
full_talbe['id'].replace(' ','')
full_talbe['Notes'] = ''

#UNIX timestamp: created
full_talbe['UNIX'] = int( time.time())



full_talbe.head(1)



In [ ]:
 #@title Save To Directory{ display-mode: "code" }
 
table = "deFi_lama_general"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
full_talbe.to_csv(path+ table)

In [ ]:
@title Dataset Stats { display-mode: "code" }
full_talbe.info()

In [ ]:
#@title Load to SQLite { display-mode: "code" }

full_talbe.to_sql(table, conn , if_exists='replace', index=False) # writes to file

 # Test Connection
df = pd.read_sql('select * from {}'.format(table), conn)

df.head()

In [ ]:
full_talbe.describe()

In [ ]:
#@title Dataset Stats { display-mode: "code" }
full_talbe.info(verbose=True)

 ## Total Combined TVL

In [ ]:
  #@title API call { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table

import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt

path = "https://api.llama.fi/charts"
#slug = 'aave' 
#General Information table
response = requests.get(path)
output = response.text
DeFi = json.loads(output)
totalLiq_talbe = pd.json_normalize(DeFi)
totalLiq_talbe.head()



In [ ]:
 #@title Save To DeFi Lama Dataset Directory{ display-mode: "code" }
 
table = "deFi_lama_total_TVL"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
totalLiq_talbe.to_csv(path+ table)

In [ ]:
#@title Load General Information Data to SQLite { display-mode: "code" }


totalLiq_talbe.to_sql(table, conn , if_exists='replace', index=False) # writes to file

 # Test Connection
df = pd.read_sql('select * from {}'.format(table), conn)

df.head()

In [ ]:
conn.close()

# Feature Engineering